In [5]:
import yfinance as yf
import pandas as pd

# -------- USER INPUT --------
stock = "SBIN.NS"
entry_time = "09:30"
exit_time  = "15:00"
# ----------------------------

# Download last 7 days 1-minute data
data = yf.download(
    tickers=stock,
    period="7d",
    interval="1m",
    auto_adjust=True
)

if data.empty:
    print("No data found.")
    exit()

# Add date and time columns
data["Date"] = data.index.date
data["Time"] = data.index.strftime("%H:%M")

results = []

unique_days = sorted(data["Date"].unique())

for day in unique_days:
    day_df = data[data["Date"] == day]

    # -------- Find entry candle --------
    # First candle AFTER or AT entry_time
    entry_row = day_df[day_df["Time"] >= entry_time].head(1)

    # -------- Find exit candle --------
    # Last candle BEFORE or AT exit_time
    exit_row = day_df[day_df["Time"] <= exit_time].tail(1)

    if entry_row.empty or exit_row.empty:
        continue

    entry_price = entry_row["Open"].iloc[0].item()
    exit_price = exit_row["Close"].iloc[0].item()

    pnl = exit_price - entry_price

    results.append([day, entry_price, exit_price, pnl])


df = pd.DataFrame(results, columns=["Date", "Entry Price", "Exit Price", "P&L"])

print("\n=== Daily Intraday P&L ===")
print(df)

print("\n=== Total P&L ===")
print("TOTAL P&L =", df["P&L"].sum())


[*********************100%***********************]  1 of 1 completed


=== Daily Intraday P&L ===
         Date  Entry Price  Exit Price       P&L
0  2025-11-12   958.950012  953.099976 -5.850037
1  2025-11-13   954.750000  954.500000 -0.250000
2  2025-11-14   963.400024  967.549988  4.149963
3  2025-11-17   973.150024  973.000000 -0.150024
4  2025-11-18   972.900024  971.900024 -1.000000
5  2025-11-19   983.250000  982.750000 -0.500000

=== Total P&L ===
TOTAL P&L = -3.60009765625
